# PITT and GITT example

This notebook deals with potentiostatic intermittent titration technique (PITT) and galvanostatic intermittent titration technique (GITT) methods. 

The measurement times are actually much too short for meaningful PITT and GITT measurements. The main focus is to describe the programming interface for the PITT and GITT.

In [2]:
from zahner_potentiostat.scpi_control.searcher import SCPIDeviceSearcher
from zahner_potentiostat.scpi_control.serial_interface import SerialCommandInterface, SerialDataInterface
from zahner_potentiostat.scpi_control.control import *
from zahner_potentiostat.scpi_control.datahandler import DataManager
from zahner_potentiostat.scpi_control.datareceiver import TrackTypes
from zahner_potentiostat.display.dcplot import DCPlot
from zahner_potentiostat.display.onlinedisplay import OnlineDisplay
from zahner_potentiostat.drivecycle.cycle_importer import getNormalisedCurrentTableForNYCCCOL

from jupyter_utils import executionInNotebook, notebookCodeToPython

if __name__ == '__main__':
    deviceSearcher = SCPIDeviceSearcher()
    deviceSearcher.searchZahnerDevices()
    commandSerial, dataSerial = deviceSearcher.selectDevice()
    ZahnerPP2x2 = SCPIDevice(SerialCommandInterface(commandSerial), SerialDataInterface(dataSerial))
    ZahnerPP2x2.clearState()

COM10:
ZAHNER-ELEKTRIK,PP212,33000,1.0.0 binary

COM9:
ZAHNER-ELEKTRIK,PP212,33000,1.0.0



# Setting general parameters
First of all, general parameters are set, which will be used in all primitives.  
The current range is held in the third shunt, as no more than 2 A current will flow.

In [ ]:
    ZahnerPP2x2.setRaiseOnErrorEnabled(True)
    ZahnerPP2x2.calibrateOffsets()
    ZahnerPP2x2.setSamplingFrequency(5)
    
    ZahnerPP2x2.setAutorangingEnabled(False)
    ZahnerPP2x2.setInterpolationEnabled(False)
    
    ZahnerPP2x2.setShuntIndex(2)
    ZahnerPP2x2.setMinimumShuntIndex(2)
    ZahnerPP2x2.setMaximumShuntIndex(2)
    
    ZahnerPP2x2.setVoltageRangeIndex(0)

# Starting the live data display

With the following command, a plotting window can be opened, in which the measured voltage and current points from the measuring device are displayed live.

The function executionInNotebook() is used to check if the execution is taking place in Jupyter notebook or not. As Jupyter cannot display the live measured data so if the execution take place in Jupyter notebook then the online display will not be executed.

In [ ]:
    onlineDisplay = None
    if executionInNotebook() == False:
        onlineDisplay = OnlineDisplay(ZahnerPP2x2.getDataReceiver())

# PITT - Potentiostatic Intermittent Titration Technique

## Setup measurement

General parameters such as sampling rate or current ranges have previously been set.

The measurement is started at OCV. Charging is carried out for a voltage of 100 mV and then discharging is carried out again 100 mV back to OCV.

In [ ]:
    ocVoltage = ZahnerPP2x2.measureOCV()
    print("open circuit reference voltage: " + str(ocVoltage) + " V")

## Execute the measurement

The [measurePITT()](http://zahner.de/documentation/zahner_potentiostat/scpi_control/control.html#zahner_potentiostat.scpi_control.control.SCPIDevice.measurePITT) method is used for the measurement. A closer look at the source code shows how the primitives were put together. An adjustment to the source code is easily possible.

In [ ]:
    ZahnerPP2x2.measurePITT(targetVoltage = ocVoltage + 0.1,
                           endVoltage = ocVoltage,
                           stepVoltage = 0.01,
                           onTime = "30s",
                           openCircuitTime = "30s",
                           startWithOCVScan = True,
                           measureOnTargetVoltage = True)

## Plot the data

This script is not executed in the Jupyter notebook because of the long execution time, since you can not see the live data.  
The plot is inserted as an image, which was created during the execution in the development environment.

In [ ]:
    dataReceiver = ZahnerPP2x2.getDataReceiver()
    dataManager = DataManager(dataReceiver)
    dataManager.saveDataAsText("PITT.txt")
    dataManager.plotTIUData("PITT.png",10,5)

![PITT](PITT.png)

## Reset configurations

Configuration is resetted for the next measurement. Here, the measured data points are deleted for the next measurement.

In [ ]:
    dataReceiver.deletePoints()

# GITT - Galvanostatic Intermittent Titration Technique

## Setup measurement

General parameters such as sampling rate or current ranges have previously been set.

The GITT measurement is also started at the OCV and it is to be charged 150 mV and then discharged to OCV minus 100 mV.

## Execute the measurement

The [measureGITT()](http://zahner.de/documentation/zahner_potentiostat/scpi_control/control.html#zahner_potentiostat.scpi_control.control.SCPIDevice.measureGITT) method is used for the measurement. A closer look at the source code shows how the primitives were put together. An adjustment to the source code is easily possible.

In [ ]:
    ZahnerPP2x2.measureGITT( targetVoltage = ocVoltage + 0.15,
                        endVoltage = ocVoltage - 0.1,
                        current = 1,
                        onTime = "30s",
                        openCircuitTime = "30s",
                        startWithOCVScan = True)

## Plot the data

This script is not executed in the Jupyter notebook because of the long execution time, since you can not see the live data.  
The plot is inserted as an image, which was created during the execution in the development environment.

In [ ]:
    dataManager.saveDataAsText("GITT.txt")
    dataManager.plotTIUData("GITT.png",10,5)

![GITT](GITT.png)

# Close the connection

Closing the online display when it has been opened and close the connection to the device.

In [ ]:
    if onlineDisplay != None:
        onlineDisplay.close()
    
    ZahnerPP2x2.close()
    print("finish")

# Deployment of the source code

**The following instruction is not needed by the user.**

It automatically extracts the pure python code from the jupyter notebook to provide it for the user.  
Thus the user does not need jupyter itself and does not have to copy the code manually.

The code is stored in a notebook-like file with the extension .py.

In [3]:
    if executionInNotebook() == True:
        notebookCodeToPython("PITTandGITT.ipynb")